In [ ]:
# import libraries

import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Read the CSV file with a maximum of 1,227,516 rows
player_21 = pd.read_csv( '/content/drive/MyDrive/Colab Notebooks/Mid-Sem Project/players_21.csv')

player_22 = pd.read_csv( '/content/drive/MyDrive/Colab Notebooks/Mid-Sem Project/players_22.csv')

# to display all columns of the data set
pd.set_option('display.max_columns', None)
player_21 = player_21.drop(columns=['sofifa_id'])
player_22 = player_22.drop(columns=['sofifa_id'])

<ipython-input-157-9427856965d7>:4: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  player_22 = pd.read_csv( '/content/drive/MyDrive/Colab Notebooks/Mid-Sem Project/players_22.csv')


In [ ]:
player_21.head()

,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,1987-06-24,170,72,241.0,FC Barcelona,Spain Primera Division,1.0,CAM,10.0,NaN,2004-07-01,2021.0,52,Argentina,1369.0,RW,10.0,Left,4,4,5,Medium/Low,Unique,Yes,138400000.0,"#Dribbler, #Distance Shooter, #FK Specialist, ...","Finesse Shot, Long Shot Taker (AI), Speed Drib...",85.0,92.0,91.0,95.0,38.0,65.0,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,32,35,24,6,11,15,14,8,NaN,89+3,89+3,89+3,92,93,93,93,92,93,93,93,91+2,87+3,87+3,87+3,91+2,66+3,65+3,65+3,65+3,66+3,62+3,52+3,52+3,52+3,62+3,19+3,https://cdn.sofifa.net/players/158/023/21_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,1985-02-05,187,83,45.0,Juventus,Italian Serie A,1.0,LS,7.0,NaN,2018-07-10,2022.0,38,Portugal,1354.0,LS,7.0,Right,4,5,5,High/Low,Unique,Yes,75900000.0,"#Aerial Threat, #Dribbler, #Distance Shooter, ...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",89.0,93.0,81.0,89.0,35.0,77.0,84,95,90,82,86,88,81,76,77,92,87,91,87,95,71,94,95,84,78,93,63,29,95,82,84,95,28,32,24,7,11,15,14,11,NaN,91+1,91+1,91+1,89,91,91,91,89,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,54+3,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/21_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
2,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,1988-08-21,184,80,21.0,FC Bayern München,German 1. Bundesliga,1.0,ST,9.0,NaN,2014-07-01,2023.0,37,Poland,NaN,NaN,NaN,Right,4,4,4,High/Medium,Unique,Yes,132000000.0,"#Distance Shooter, #Clinical Finisher","Solid Player, Finesse Shot, Outside Foot Shot,...",78.0,91.0,78.0,85.0,43.0,82.0,71,94,85,84,89,85,79,85,70,88,77,78,77,93,82,89,84,76,86,85,81,49,94,79,88,88,35,42,19,15,6,12,8,10,NaN,89+2,89+2,89+2,85,87,87,87,85,85+3,85+3,85+3,83+3,79+3,79+3,79+3,83+3,64+3,65+3,65+3,65+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/21_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,NaN,https://cdn.sofifa.net/flags/pl.png
3,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW,

In [ ]:
player_22.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,1987-06-24,170,72,73.0,Paris Saint-Germain,French Ligue 1,1.0,RW,30.0,NaN,2021-08-10,2023.0,52,Argentina,1369.0,RW,10.0,Left,4,4,5,Medium/Low,Unique,Yes,144300000.0,"#Dribbler, #Distance Shooter, #FK Specialist, ...","Finesse Shot, Long Shot Taker (AI), Playmaker ...",85.0,92.0,91.0,95.0,34.0,65.0,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,20,35,24,6,11,15,14,8,NaN,89+3,89+3,89+3,92,93,93,93,92,93,93,93,91+2,87+3,87+3,87+3,91+2,66+3,64+3,64+3,64+3,66+3,61+3,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,1988-08-21,185,81,21.0,FC Bayern München,German 1. Bundesliga,1.0,ST,9.0,NaN,2014-07-01,2023.0,37,Poland,1353.0,RS,9.0,Right,4,4,5,High/Medium,Unique,Yes,197200000.0,"#Aerial Threat, #Distance Shooter, #Clinical F...","Solid Player, Finesse Shot, Outside Foot Shot,...",78.0,92.0,79.0,86.0,44.0,82.0,71,95,90,85,89,85,79,85,70,88,77,79,77,93,82,90,85,76,86,87,81,49,95,81,90,88,35,42,19,15,6,12,8,10,NaN,90+2,90+2,90+2,85,88,88,88,85,86+3,86+3,86+3,84+3,80+3,80+3,80+3,84+3,64+3,66+3,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,1985-02-05,187,83,11.0,Manchester United,English Premier League,1.0,ST,7.0,NaN,2021-08-27,2023.0,38,Portugal,1354.0,ST,7.0,Right,4,5,5,High/Low,Unique,Yes,83300000.0,"#Aerial Threat, #Dribbler, #Distance Shooter, ...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",87.0,94.0,80.0,88.0,34.0,75.0,87,95,90,80,86,88,81,84,77,88,85,88,86,94,74,94,95,77,77,93,63,29,95,76,88,95,24,32,24,7,11,15,14,11,NaN,90+1,90+1,90+1,88,89,89,89,88,86+3,86+3,86+3,86+3,78+3,78+3,78+3,86+3,63+3,59+3,59+3,59+3,63+3,60+3,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png


In [ ]:
player_21.describe()

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_jersey_number,club_contract_valid_until,nationality_id,nation_team_id,nation_jersey_number,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,18944.000000,18944.000000,18944.000000,1.870700e+04,18719.000000,18944.000000,18944.000000,18944.000000,18719.000000,18719.000000,18719.000000,18719.000000,18944.000000,1127.000000,1127.000000,18944.000000,18944.000000,18944.000000,1.794900e+04,16861.000000,16861.000000,16861.000000,16861.000000,16861.000000,16861.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,2083.000000
mean,226232.328917,65.677787,71.086729,2.902288e+06,9148.482825,25.225823,181.190773,75.016892,47908.905551,1.355468,20.589668,2021.983332,57.107791,32631.673469,12.029281,2.936603,2.363017,1.091850,4.296353e+06,67.668110,52.274954,57.139434,62.455430,51.316292,64.458988,49.612859,45.796083,51.874578,58.705448,42.669552,55.546400,47.187289,42.356155,52.653030,58.482791,64.285103,64.326489,63.333193,61.609428,63.918549,57.752798,64.590530,62.596020,64.742927,46.758763,55.488862,46.250000,50.255807,53.828231,48.050412,57.978674,46.470017,47.581767,45.546505,16.446052,16.236486,16.103357,16.225982,16.551309,37.203553
std,27166.574284,7.002278,6.109985,7.743775e+06,19893.149956,4.697354,6.825672,7.057140,53906.733776,0.739015,17.057568,1.260426,48.696159,49455.505830,6.674627,0.667132,0.766469,0.361841,1.005968e+07,10.984862,13.991421,10.273655,10.048706,16.405179,9.746706,18.153057,19.592269,17.321709,14.572383,17.661569,18.787112,18.231204,17.239561,15.205281,16.598040,14.926452,14.689422,14.625189,9.112954,14.075215,13.330631,11.888479,15.878793,12.519775,19.305319,17.203045,20.738106,19.443738,13.724072,15.671721,12.118390,20.169591,21.402461,20.953997,17.577332,16.845480,16.519399,17.017341,17.878121,10.714523
min,41.000000,47.000000,47.000000,9.000000e+03,500.000000,16.000000,155.000000,50.000000,1.000000,1.000000,1.000000,2020.000000,1.000000,1318.000000,1.000000,1.000000,1.000000,1.000000,9.000000e+03,25.000000,16.000000,25.000000,25.000000,15.000000,28.000000,6.000000,3.000000,5.000000,7.000000,3.000000,5.000000,4.000000,5.000000,5.000000,5.000000,13.000000,12.000000,14.000000,24.000000,12.000000,18.000000,15.000000,12.000000,16.000000,4.000000,9.000000,3.000000,2.000000,9.000000,6.000000,12.000000,3.000000,5.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000
25%,210028.750000,61.000000,67.000000,4.750000e+05,1000.000000,21.000000,176.000000,70.000000,461.000000,1.000000,9.000000,2021.000000,21.000000,1352.000000,6.000000,3.000000,2.000000,1.000000,5.250000e+05,62.000000,42.000000,50.000000,57.000000,36.000000,58.000000,38.000000,30.000000,44.000000,54.000000,30.000000,49.000000,35.000000,31.000000,43.000000,54.000000,57.000000,57.000000,55.000000,56.00

In [ ]:
player_22.describe()

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_jersey_number,club_contract_valid_until,nationality_id,nation_team_id,nation_jersey_number,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,19239.000000,19239.000000,19239.000000,1.916500e+04,19178.000000,19239.000000,19239.000000,19239.000000,19178.000000,19178.000000,19178.000000,19178.000000,19239.000000,759.000000,759.000000,19239.000000,19239.000000,19239.000000,1.806300e+04,17107.000000,17107.000000,17107.000000,17107.000000,17107.000000,17107.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,2132.000000
mean,231468.086959,65.772182,71.079370,2.850452e+06,9017.989363,25.210822,181.299704,74.943032,50580.498123,1.354364,20.945250,2022.764000,58.602682,14480.848485,12.567852,2.946151,2.352461,1.094184,5.374044e+06,68.213071,52.345297,57.312562,62.561174,51.703630,64.823289,49.577421,45.894433,51.783877,58.867977,42.463849,55.660429,47.268933,42.249025,53.072249,58.472010,64.652893,64.714902,63.500078,61.450023,64.068611,57.776860,64.813504,63.084880,65.007745,46.642705,55.538957,46.613545,50.330215,53.964603,47.858724,57.929830,46.601746,48.045584,45.906700,16.406102,16.192474,16.055356,16.229274,16.491814,36.439962
std,27039.717497,6.880232,6.086213,7.613700e+06,19470.176724,4.748235,6.863179,7.069434,54401.868535,0.747865,17.909369,1.213203,50.298614,35328.730217,7.039116,0.671560,0.767659,0.371098,1.494837e+07,10.933155,14.051623,10.068965,9.651312,16.189746,9.791886,18.034661,19.721023,17.294183,14.490858,17.653329,18.784590,18.181085,17.178590,15.026569,16.663722,15.167399,14.965426,14.862285,9.042281,14.324789,13.192224,12.122977,16.145279,12.663518,19.411583,16.972181,20.677077,19.621601,13.650481,15.768583,12.159326,20.200807,21.232718,20.755683,17.574028,16.839528,16.564554,17.059779,17.884833,10.751563
min,41.000000,47.000000,49.000000,9.000000e+03,500.000000,16.000000,155.000000,49.000000,1.000000,1.000000,1.000000,2021.000000,1.000000,1318.000000,1.000000,1.000000,1.000000,1.000000,1.600000e+04,28.000000,18.000000,25.000000,27.000000,14.000000,29.000000,6.000000,2.000000,5.000000,7.000000,3.000000,4.000000,6.000000,4.000000,9.000000,8.000000,14.000000,15.000000,18.000000,25.000000,15.000000,20.000000,22.000000,12.000000,19.000000,4.000000,10.000000,3.000000,2.000000,10.000000,7.000000,12.000000,4.000000,5.000000,5.000000,2.000000,2.000000,2.000000,2.000000,2.000000,15.000000
25%,214413.500000,61.000000,67.000000,4.750000e+05,1000.000000,21.000000,176.000000,70.000000,479.000000,1.000000,9.000000,2022.000000,21.000000,1338.000000,7.000000,3.000000,2.000000,1.000000,8.060000e+05,62.000000,42.000000,51.000000,57.000000,37.000000,59.000000,38.000000,30.000000,44.000000,54.000000,30.000000,50.000000,35.000000,31.000000,44.000000,55.000000,57.000000,58.000000,55.000000,56.000

In [ ]:
## Dropping irrelevant columns: Categorical features
cat_21 = player_21.select_dtypes(include=['object']).columns
cat_22 = player_22.select_dtypes(include=['object']).columns

#Skipping URLS
t = ['url' not in c for c in cat_21]
t2 = ['url' not in c for c in cat_22]

cat_21 = cat_21[t]
cat_21 = cat_22[t2]


FEATURE EXTRACTIONS FOR TEXT

In [ ]:
player_21[cat_21]

,short_name,long_name,player_positions,dob,club_name,league_name,club_position,club_loaned_from,club_joined,nationality_name,nation_position,preferred_foot,work_rate,body_type,real_face,player_tags,player_traits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,10059,10302,539,942,238,41,0,278,8,6,22,0,7,9,1,25,172,215,215,215,139,132,132,132,139,266,266,266,249,216,216,216,249,133,136,136,136,133,110,59,59,59,110,18
1,3261,3332,587,391,361,24,13,278,1135,123,12,1,1,9,1,4,699,216,216,216,137,131,131,131,137,263,263,263,246,197,197,197,246,126,110,110,110,126,104,66,66,66,104,20
2,14329,14687,560,1271,240,19,27,278,289,122,26,1,2,9,1,15,805,214,214,214,133,128,128,128,133,254,254,254,233,187,187,187,233,119,136,136,136,119,104,96,96,96,104,18
3,13007,12964,331,2438,461,17,14,278,888,20,26,1,2,9,1,66,343,206,206,206,138,130,130,130,138,265,265,265,248,206,206,206,248,139,116,116,116,139,110,52,52,52,110,20
4,8987,9626,10,2229,407,15,19,278,487,14,17,1,0,9,1,33,355,203,203,203,136,129,129,129,136,264,264,264,247,217,217,217,247,204,232,232,232,204,185,158,158,158,185,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,17684,18137,55,4982,138,6,28,278,1769,30,26,1,4,4,0,75,915,20,20,20,16,15,15,15,16,15,15,15,19,21,21,21,19,39,39,39,39,39,44,46,46,46,44,9
18940,6680,18648,149,4875,560,6,21,278,1769,30,26,1,4,1,0,75,915,35,35,35,31,29,29,29,31,46,46,46,44,49,49,49,44,48,52,52,52,48,49,44,44,44,49,9
18941,15978,18041,149,2366,608,6,21,278,1769,30,26,1,4,2,0,75,915,40,40,40,33,31,31,31,33,48,48,48,52,48,48,48,52,46,49,49,49,46,48,43,43,43,48,1
18942,16799,7152,560,6175,444,14,28,278,1769,48,26,1,8,1,0,75,915,43,43,43,35,34,34,34,35,53,53,53,48,42,42,42,48,30,21,21,21,30,26,10,10,10,26,7


In [ ]:
### Impute missing values in

## players_21:
# Extract numerical features
num_21 = player_21.select_dtypes(include=['int64', 'float64']).columns
# Impute missing values in players_21
num_imputer_21 = SimpleImputer(strategy='mean')

player_21[num_21] = num_imputer_21.fit_transform(player_21[num_21])

## players_22:
# Extract numerical features
num_22 = player_22.select_dtypes(include=['int64', 'float64']).columns
# Impute missing values in players_22
num_imputer_22 = SimpleImputer(strategy='mean')
player_22[num_22] = num_imputer_22.fit_transform(player_22[num_22])


#Filling in NaNs for text values (imputing with forward fill)
player_21[cat_21].fillna(method='ffill', inplace=True)
player_22[cat_22].fillna(method='ffill', inplace=True)


#coverting texts into numerical values with encoder
enc = LabelEncoder()
for c in player_21[cat_21]:
  player_21[c] = enc.fit_transform(player_21[c])


enc2 = LabelEncoder()
for c2 in player_22[cat_22]:
  player_22[c2] = enc.fit_transform(player_22[c2])

# Merging textual and numerical data for training
merge_num_text_21 = pd.concat([player_21[num_21], player_21[cat_21]], axis=0)
merge_num_text_22 = pd.concat([player_22[num_22], player_22[cat_22]], axis=0)


<ipython-input-160-d951763567e5>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_21[cat_21].fillna(method='ffill', inplace=True)
<ipython-input-160-d951763567e5>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_22[cat_22].fillna(method='ffill', inplace=True)


In [ ]:
# nanValues = merge_num_text_21.isna()
# merge_num_text_21= merge_num_text_21[nanValues]

Using correlation to find the dependent variables that are strongly contributing to the players overall
INDEPENDENT VARIABLE: Overall
DEPENDENT: Remaining variables


In [ ]:
corr_matrix = merge_num_text_21.corr()
corr_matrix_22 = player_22.corr()
corr_matrix



,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_jersey_number,club_contract_valid_until,nationality_id,nation_team_id,nation_jersey_number,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,short_name,long_name,player_positions,dob,club_name,league_name,club_position,club_loaned_from,club_joined,nationality_name,nation_position,preferred_foot,work_rate,body_type,real_face,player_tags,player_traits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
overall,1.000000,0.636366,0.555192,0.589060,0.468197,0.031579,0.146242,-0.207319,-0.215148,-0.187180,0.122961,-0.098959,-0.071494,-0.014796,0.222814,0.381369,0.440987,0.614344,0.188862,0.454391,0.662090,0.596558,0.333616,0.493539,0.410530,0.325413,0.327239,0.502191,0.374330,0.378455,0.420495,0.385617,0.487147,0.449372,0.206962,0.215745,0.279889,0.867234,0.128349,0.558372,0.282440,0.381869,0.358049,0.407525,0.401920,0.320162,0.354239,0.509087,0.328198,0.705252,0.320986,0.260325,0.232392,0.003128,0.003454,-0.000512,0.012274,0.004052,0.180002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
potential,0.636366,1.000000,0.531593,0.486206,-0.262195,-0.011113,-0.023860,-0.303501,-0.168442,0.013721,0.256409,-0.201440,-0.091713,-0.007677,0.161616,0.297293,0.308360,0.573036,0.254777,0.268612,0.411394,0.448911,0.163723,0.152303,0.257515,0.217648,0.179781,0.351071,0.230763,0.308379,0.277232,0.215330,0.314737,0.332585,0.240037,0.240678,0.236697,0.512926,0.155215,0.336309,0.113711,0.208670,0.065667,0.243391,0.190117,0.158788,0.229676,0.339285,0.196769,0.425953,0.172626,0.158619,0.145028,-0.035177,-0.034622,-0.039857,-0.033219,-0.034552,0.081052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
value_eur,0.555192,0.531593,1.000000,0.810460,0.044768,0.003779,0.034550,-0.179552,-0.121658,-0.064580,0.186589,-0.072684,-0.120344,-0.015630,0.145720,0.260922,0.532620,0.973824,0.181486,0.280831,0.386445,0.376522,0.142108,0.202626,0.229184,0.212159,0.156967,0.290918,0.236122,0.238708,0.255278,0.215751,0.273204,0.265552,0.163391,0.168024,0.178609,0.491619,0.119759,0.304725,0.119543,0.200311,0.121668,0.232296,0.174373,0.137856,0.223508,0.323443,0.191358,0.391402,0.141271,0.116499,0.099182,-0.012174,-0.011248,-0.012523,-0.010800,-0.010286,0.068224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wage_eur,0.589060,0.486206,0.810460,1.000000,0.161626,0.020718,0.063778,-0.224062,-0.137278,-0.065860,0.134670,-0.077124,-0.109541,-0.037046,0.154090,0.268903,0.610869,0.831925,0.132428,0.300367,0.415753,0.386075,0.173649,0.235402,0.254457,0.225338,0.200181,0.317372,0.260992,0.252774,0.280985,0.249550,0.306614,0.288923,0.138555,0.142081,0.170180,0.524714,0.107939,0.342131,0.144931,0.201449,0.154428,0.258459,0.219205,0.170669,0.241082,0.342663,0.227652,0.433662,0.171638,0.143803,0.125132,-0.020143,-0.020634,-0.021393,-0.

Identifying the key variables that strongly contribute to the players overall according to the correlation analysis on the dataset.

In [ ]:
corr_matrix_22['overall'].sort_values(ascending=False)
corr_matrix['overall'].sort_values(ascending=False)


overall                1.000000
movement_reactions     0.867234
mentality_composure    0.705252
passing                0.662090
potential              0.636366
                         ...   
lcb                         NaN
cb                          NaN
rcb                         NaN
rb                          NaN
gk                          NaN
Name: overall, Length: 103, dtype: float64

Identifying the key features that characterize the dependent variables. Dropping all other variables except the below variables obtained from the correlation analysis.

In [ ]:
#Threshold for correlation relevance is set to 50%
corr_matrix['overall'] > 0.5

overall       True
potential     True
value_eur     True
wage_eur      True
age          False
             ...  
lcb          False
cb           False
rcb          False
rb           False
gk           False
Name: overall, Length: 103, dtype: bool

In [ ]:
# List of feature names with highest correlation to 'overall'
selected_features = ['movement_reactions','passing','mentality_composure','dribbling','potential','release_clause_eur','wage_eur',
    'value_eur','power_shot_power','physic','mentality_vision','attacking_short_passing'
]
# Select these features from the dataset
selected_data = player_21[['overall'] + selected_features]

Presenting the Dependent Variables Alongside Their Various Metrics

Ensuring that all missing values have been effectively substituted with their respective means.

Dividing the dataset into two distinct components: the Y-variable, representing the target or dependent variable, and the X-variables, which encompass the independent variables or features.

In [ ]:
from sklearn.model_selection import train_test_split
# Specify the features (X) and the target variable (y)
y = selected_data['overall']  # Target variable
X = selected_data.drop(columns=['overall'])  # Features


Before standardizing the dependent variables

In [ ]:
X

Standardizing the dependent variables to ensure uniform measurement units across all of them.

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
scaled = sc.fit_transform(X)
X = pd.DataFrame(scaled, columns=X.columns)
X

,potential,value_eur,wage_eur,release_clause_eur,passing,dribbling,attacking_short_passing,movement_reactions,power_shot_power,mentality_vision,mentality_composure
0,3.586563,13.073165,27.857178,13.695696,3.493625,3.433013,2.216206,3.554438,2.119026,3.000047,3.137573
1,3.422893,7.809992,10.662997,7.312715,2.461857,2.800094,1.598583,3.664174,2.719163,2.052781,3.055051
2,3.259222,14.047827,11.674420,13.042079,2.152326,2.378148,1.735833,3.444701,2.344077,1.834181,2.477402
3,3.259222,16.776879,13.191553,16.565484,2.977741,3.327526,1.941707,3.225227,1.668922,2.635714,2.890009
4,3.259222,16.387015,18.248665,16.003782,3.699979,2.694607,2.422080,3.225227,2.494112,2.927180,2.724966
...,...,...,...,...,...,...,...,...,...,...,...
18939,-3.123941,-0.368070,-0.412078,-0.432955,-3.212867,-3.740069,-1.901280,-1.493455,-1.706851,-2.100615,-1.483625
18940,-2.960270,-0.368070,-0.412078,-0.431423,-0.839801,-1.630339,-0.254285,-1.273981,-1.481799,-0.060350,-1.896232
18941,-3.942295,-0.371319,-0.361507,-0.433977,-0.839801,-1.735826,-0.185661,-1.932402,-0.656610,-0.716150,-1.896232
18942,-0.668878,-0.360272,-0.437363,-0.421926,-1.768392,-0.997420,-0.940533,-0.944771,-0.581593,-0.351817,-1.071018


Create the models, initialize and train the models to get ready for prediction
Using Ensemble Learning

In [ ]:
# Split the data into training and testing sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
}

regressorGB = GradientBoostingRegressor()
grid_searchGB = GridSearchCV(estimator=regressorGB, param_grid=param_grid, cv=KFold(n_splits=5), scoring='neg_mean_squared_error')
grid_searchGB.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error')

In [ ]:
grid_searchGB.best_score_

-1.2604342642062178

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

regressorRF = RandomForestRegressor()
grid_searchRF = GridSearchCV(estimator=regressorRF, param_grid=param_grid, cv=KFold(n_splits=5), scoring='neg_mean_squared_error')
grid_searchRF.fit(X_train, y_train)

AttributeError: ignored

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
}

regressorXGB = xgb.XGBRegressor()
grid_searchXGB = GridSearchCV(estimator=regressorXGB, param_grid=param_grid, cv=KFold(n_splits=5), scoring='neg_mean_squared_error')
grid_searchXGB.fit(X_train, y_train)


In [ ]:
best_model = grid_searchGB
best_params = grid_searchGB.get_params

if grid_searchGB.best_score_ > grid_searchRF.best_score_ and grid_searchGB.best_score_ > grid_searchXGB.best_score_:
    best_model = grid_searchGB
    best_params = grid_searchGB.best_params_
elif grid_searchRF.best_score_ > grid_searchGB.best_score_ and grid_searchRF.best_score_ > grid_searchXGB.best_score_:
    best_model = grid_searchRF
    best_params = grid_searchRF.best_params_
else:
    best_model = grid_searchXGB
    best_params = grid_searchXGB.best_params_


SAVING THE MODEL AS A PICKLE FILE

In [ ]:
import pickle

# Save the model to a file
with open('football_prediction_model.pkl', 'wb') as model_file:
    pickle.dump(best_model, model_file)
